In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import json # library to handle JSON files

#import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
#%matplotlib inline

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import bs4 as bs
import urllib.request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# geocoder
!conda install -c conda-forge geocoder --yes 
import geocoder

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py36_0   conda-forge
    ratelim:    0.1.6-py36_0 conda-forge


orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB     | ##################################### | 100% 
ratelim-0.1.6      

In [2]:
# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [3]:
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [4]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
# More than one neighborhood can exist in one postal code area.
df = df.groupby(['PostalCode', 'Borough']).agg(', '.join)
df = df.reset_index()
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighbourhood']=='Not assigned', ['Neighbourhood']] = 'Queen\'s Park'
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


What I have done so far on the data cleaning above?
1. Import and scrap wikipage using BeautifulSoup package.
2. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
3. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
4. The rows of same PostalCode and Borough will be combined into one row with the neighborhoods separated with a comma.
5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

In [8]:
df['Latitude'] = ''
df['Longitude'] = ''

# loop through the postal code to find out the latitude and longitude
for index, data in df.iterrows():
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(data['PostalCode']))
        lat_lng_coords = g.latlng
    data['Latitude'] = lat_lng_coords[0]
    data['Longitude'] = lat_lng_coords[1]
    #print ('PostalCode:', data['PostalCode'], 'Latitude:', data['Latitude'], 'Longitude:', data['Longitude'])

In [10]:
# print out the updated dataframe
df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.6915,-79.2866
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


In [11]:
# double check on the dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


For above, I have added the latitude and longtitude for each postal code in the dataframe.<br><br><b>Project Part 3: Segmenting and Clustering the neighbourhoods in Toronto.</b><br><br>1. For this project, I shall focus on the Borough of Scarborough of Toronto.<br><br>2. I shall be using K-Clustering to find out the most common venues and then segment and group them accordingly.<br><br>3. Only the Top 5 clusters will be displayed for the ease of this project purposes.<br><br>

In [12]:
# Use geopy library to get the latitude and longitude values of Toronto Ontario
address = 'Toronto Ontario, TO'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto Ontario are 43.638093, -79.4665843.


In [13]:
# create map of Toronto Ontario using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [14]:
# just focus on Scarborough
scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.6915,-79.2866
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


In [15]:
address = 'Scarborough, Toronto Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [16]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [17]:
# Foursquare Credentials
CLIENT_ID = 'RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB' # your Foursquare ID
CLIENT_SECRET = '2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB
CLIENT_SECRET:2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ


In [18]:
# explore the first neighborhood in our dataframe.
scarborough_data.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [19]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.8066863, -79.1943534.


In [20]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RF2HLDBFJ0Z0IAOHHOTQG4CMUCDM0RGVAF3DVCQFD153FMOB&client_secret=2LYBEQRR5TXMPQWUFKMLWCIKWVOA2G3U3Y1YCYL20MWXKRHJ&v=20180605&ll=43.8066863,-79.1943534&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ba74e1cf594df05c5f3fe74'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.1881295807304},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926959}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'dis

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [25]:
# Explore Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# get scarborough venues
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

print(scarborough_venues.shape)
scarborough_venues.head(10)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood]], Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
(114, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood]], Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood]], Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood]], Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood]], Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764042,-79.193371,Rental Car Location
7,"Guildwood]], Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood]], Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


In [27]:
# find out how many unique categories
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",27,27,27,27,27,27
"Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West",4,4,4,4,4,4
"Dorset Park, Scarborough Town Centre, Wexford Heights",8,8,8,8,8,8
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


In [28]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 68 uniques categories.


In [29]:
# analyze each neighborhood

# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Beer Store,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fried Chicken Joint,Gas Station,General Entertainment,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Hobby Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Moving Target,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Sporting Goods Shop,Supermarket,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood]], Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood]], Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
scarborough_onehot.shape

(114, 69)

In [31]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Beer Store,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fried Chicken Joint,Gas Station,General Entertainment,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Hobby Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Moving Target,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Sporting Goods Shop,Supermarket,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant
0,Agincourt,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.250000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000,0.25,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.0,0.0,0.000000,0.333333,0.000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
2,"Birch Cliff, Cliffside West",0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.25,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
3,Cedarbrae,0.000,0.00,0.142857,0.000,0.142857,0.142857,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000
4,"Clairlea, Golden Mile, Oakridge",0.000,0.00,0.000000,0.000,0.037037,0.037037,0.0,0.037037,0.000000,0.000,0.000000,0.037037,0.00,0.037037,0.000000,0.000000,0.074074,0.00,0.000000,0.037037,0.000000,0.037037,0.037037,0.000,0.000000,0.037037,0.00,0.037037,0.148148,0.037037,0.037037,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.037037,0.00,0.000000,0.037037,0.000000,0.000,0.00,0.00,0.0,0.0,0.037037,0.000000,0.000,0.000000,0.037037,0.000000,0.000000,0.000000,0.037037,0.000,0.00,0.000,0.037037,0.037037,0.000000,0.000000,0.037037,0.037037,0.000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.100000,0.00,0.000000,0.000000,0.000000,

In [32]:
scarborough_grouped.shape
#scarborough_grouped.head(10)

(17, 69)

In [33]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
               venue  freq
0       Skating Rink  0.25
1             Lounge  0.25
2     Sandwich Place  0.25
3     Breakfast Spot  0.25
4  Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                           venue  freq
0                           Park  0.33
1                    Coffee Shop  0.33
2                     Playground  0.33
3                      Pet Store  0.00
4  Paper / Office Supplies Store  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2                   Café  0.25
3           Skating Rink  0.25
4                 Lounge  0.00


----Cedarbrae----
                 venue  freq
0     Hakka Restaurant  0.14
1   Athletics & Sports  0.14
2               Bakery  0.14
3                 Bank  0.14
4  Fried Chicken Joint  0.14


----Clairlea, Golden Mile, Oakridge----
                  venue  freq
0         Grocery Store  0.15

In [34]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Gym,Grocery Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Chinese Restaurant
2,"Birch Cliff, Cliffside West",College Stadium,Café,General Entertainment,Skating Rink,Fish & Chips Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant
3,Cedarbrae,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Bakery,Bank,Electronics Store,Fast Food Restaurant,Filipino Restaurant
4,"Clairlea, Golden Mile, Oakridge",Grocery Store,Coffee Shop,Bus Line,Gym,Gym / Fitness Center,Gas Station,Video Store,Filipino Restaurant,Fast Food Restaurant,Liquor Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Coffee Shop,Thai Restaurant,Italian Restaurant,Fried Chicken Joint,Shopping Mall,Chinese Restaurant,Fast Food Restaurant,Electronics Store
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Movie Theater,Motel,Skating Rink,Vietnamese Restaurant,Fish & Chips Shop,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Brewery,Latin American Restaurant,Fish & Chips Shop,Pet Store,Chinese Restaurant,Bar,Beer Store,American Restaurant
8,"East Birchmount Park, Ionview, Kennedy Park",Hobby Shop,Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Train Station,Auto Garage,Gym / Fitness Center,Gym,Grocery Store
9,"Guildwood]], Morningside, West Hill",Breakfast Spot,Electronics Store,Mexican Restaurant,Pizza Place,Rental Car Location,Medical Center,Filipino Restaurant,Discount Store,Fast Food Restaurant,Vietnamese Restaurant


In [36]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944,0,Fast Food Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Department Store,Discount Store,Electronics Store,Filipino Restaurant,Fish & Chips Shop,Gas Station,Video Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605,2,Moving Target,Bar,Vietnamese Restaurant,Fried Chicken Joint,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Gas Station
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill",43.7636,-79.1887,0,Breakfast Spot,Electronics Store,Mexican Restaurant,Pizza Place,Rental Car Location,Medical Center,Filipino Restaurant,Discount Store,Fast Food Restaurant,Vietnamese Restaurant
3,M1G,Scarborough,Woburn,43.771,-79.2169,0,Coffee Shop,Korean Restaurant,Fried Chicken Joint,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Gas Station
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,0,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Bakery,Bank,Electronics Store,Fast Food Restaurant,Filipino Restaurant


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# Cluster 1 
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Department Store,Discount Store,Electronics Store,Filipino Restaurant,Fish & Chips Shop,Gas Station,Video Store
2,Scarborough,0,Breakfast Spot,Electronics Store,Mexican Restaurant,Pizza Place,Rental Car Location,Medical Center,Filipino Restaurant,Discount Store,Fast Food Restaurant,Vietnamese Restaurant
3,Scarborough,0,Coffee Shop,Korean Restaurant,Fried Chicken Joint,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Gas Station
4,Scarborough,0,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Bakery,Bank,Electronics Store,Fast Food Restaurant,Filipino Restaurant
5,Scarborough,0,Health & Beauty Service,Playground,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Vietnamese Restaurant
6,Scarborough,0,Hobby Shop,Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Train Station,Auto Garage,Gym / Fitness Center,Gym,Grocery Store
7,Scarborough,0,Grocery Store,Coffee Shop,Bus Line,Gym,Gym / Fitness Center,Gas Station,Video Store,Filipino Restaurant,Fast Food Restaurant,Liquor Store
8,Scarborough,0,American Restaurant,Movie Theater,Motel,Skating Rink,Vietnamese Restaurant,Fish & Chips Shop,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant
9,Scarborough,0,College Stadium,Café,General Entertainment,Skating Rink,Fish & Chips Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant
11,Scarborough,0,Vietnamese Restaurant,Smoke Shop,Auto Garage,Breakfast Spot,Middle Eastern Restaurant,Sandwich Place,Shopping Mall,Accessories Store,Skating Rink,Sporting Goods Shop


In [42]:
# Cluster 2
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,1,Pizza Place,Noodle House,Coffee Shop,Thai Restaurant,Italian Restaurant,Fried Chicken Joint,Shopping Mall,Chinese Restaurant,Fast Food Restaurant,Electronics Store


In [43]:
# Cluster 3
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Moving Target,Bar,Vietnamese Restaurant,Fried Chicken Joint,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Gas Station
16,Scarborough,2,Indian Restaurant,Grocery Store,Burger Joint,Coffee Shop,Sandwich Place,Restaurant,Vietnamese Restaurant,Filipino Restaurant,Department Store,Discount Store


In [44]:
# Cluster 4
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3,Park,Playground,Coffee Shop,Gym,Grocery Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Chinese Restaurant


In [45]:
# Cluster 5
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,4,Indian Restaurant,Vietnamese Restaurant,Brewery,Latin American Restaurant,Fish & Chips Shop,Pet Store,Chinese Restaurant,Bar,Beer Store,American Restaurant


<b>Final Analysis and Observation<br><br></b>Above marked my completion to explore and cluster the neighborhoods in the borough of Scarborough in Toronto Ontario. After examining all the 5 resultant clusters, the features that make each cluster different can be grouped. It has been concluded that each cluster posses a certain type of venues that make them different from the others in the way they can therefore can group together. By looking at each cluster's characteristics, I have the following findings:<br><br>Cluster 1: The Mixed Retail cluster. It has a good mixed of restaurants and other retail shops for the 1st few most common venues in this neighbourhood for this cluster.<br><br>Cluster 2: The Restaurant cluster. Here the 1st few most common venues in this cluster are mostly made up of food related shops such as restaurant.<br><br>Cluster 3: The Food & Commercial cluster. Here the 1st few most common venues in this cluster are mostly made up of food and other shops as a common sights for the neighbourhood.<br><br>Cluster 4: The Sports & Activities cluster. Here the 1st few most common venues in this cluster are mostly made up of children, sports or other activities related venues such as playground.<br><br>Cluster 5: The Wine & Bar cluster. Here the 1st few most common venues in this cluster are mostly made up of brewery, wine, bar or related venues.<br><br>